In [1]:
"""
-> For the Project we'll be using OfficeDataProject.csv
-> Read data from the file in the DF and perform following analytiics with it:
    :) Print the total number of employees in the company
    :) Print the total number of departments in the company()
    :)Print the department names of the company
    :) Print the total number of employees in each department
    :)Print the total number of employees in each state
    :) Print the total number of employees in each state in each department
    :) Print the minimum and maximum salaries in each department and sort salaries in ascending order
    :) Print the name of employees working in NY stae under Finance department whose bonuses are greater than \
        an average bonuses of employees in NY state.
    :) Raise the salaries $500 of all employees whose age is greater than 45.
    :) Create DF of all those employees whose age is greater than 45 and save them in a file.
"""

"\n-> For the Project we'll be using OfficeDataProject.csv\n-> Read data from the file in the DF and perform following analytiics with it:\n    :) Print the total number of employees in the company\n    :) Print the total number of departments in the company()\n    :)Print the department names of the company\n    :) Print the total number of employees in each department\n    :)Print the total number of employees in each state\n    :) Print the total number of employees in each state in each department\n    :) Print the minimum and maximum salaries in each department and sort salaries in ascending order\n    :) Print the name of employees working in NY stae under Finance department whose bonuses are greater than         an average bonuses of employees in NY state.\n    :) Raise the salaries $500 of all employees whose age is greater than 45.\n    :) Create DF of all those employees whose age is greater than 45 and save them in a file.\n"

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf, count, min, max, avg, mean
from pyspark.sql.types import IntegerType

In [4]:
spark = SparkSession.builder.appName("spark_mini_project").getOrCreate()
spark

In [5]:
df = spark.read.options(header = True, inferSchema = True).csv("OfficeDataProject.csv")
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

In [10]:
# 1. Print the total number of Employees in the company
n_employee = df.select(df.employee_name).distinct().count()
print(f" Total Number of Employees in the Company = {n_employee}")

 Total Number of Employees in the Company = 939


In [15]:
# 2. Print the total number of departments in the company()
n_department = df.select(col("department")).distinct().count()
print(f"Total Number of departments in the comapny = {n_department}")

Total Number of departments in the comapny = 6


In [16]:
# 3. Print the department names of the company
df.select(col("department")).distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [26]:
# 4.  Print the total number of employees in each department

df.groupBy("department").agg(count('*').alias('Total_Employee_per_department')).show()

+----------+-----------------------------+
|department|Total_Employee_per_department|
+----------+-----------------------------+
|     Sales|                          169|
|        HR|                          171|
|   Finance|                          162|
|Purchasing|                          166|
| Marketing|                          170|
|  Accounts|                          162|
+----------+-----------------------------+



In [25]:
# 5. Print the total number of employees in each state
df.groupBy("state").agg(count('*').alias("Employee_per_state")).show()

+-----+------------------+
|state|Employee_per_state|
+-----+------------------+
|   LA|               205|
|   CA|               205|
|   WA|               208|
|   NY|               173|
|   AK|               209|
+-----+------------------+



In [30]:
# 6. Print the total number of employees in each state in each department
df.groupBy("state","department").agg(count('*').alias("Employye_per_state_per_department"))\
.sort(col("state"), col("Employye_per_state_per_department")).show()

+-----+----------+---------------------------------+
|state|department|Employye_per_state_per_department|
+-----+----------+---------------------------------+
|   AK|        HR|                               25|
|   AK|Purchasing|                               30|
|   AK|  Accounts|                               37|
|   AK|   Finance|                               37|
|   AK|     Sales|                               38|
|   AK| Marketing|                               42|
|   CA|        HR|                               28|
|   CA|Purchasing|                               32|
|   CA| Marketing|                               33|
|   CA|   Finance|                               35|
|   CA|  Accounts|                               35|
|   CA|     Sales|                               42|
|   LA| Marketing|                               26|
|   LA|  Accounts|                               29|
|   LA|   Finance|                               29|
|   LA|     Sales|                            

In [41]:
# 7. Print the minimum and maximum salaries in each department and sort salaries in ascending order
df.groupBy("department").agg(min("salary")\
                             .alias("Min_Salary"), max("salary").alias("Max_Salary")).sort(col("Min_Salary"),col("Max_Salary")).show()

+----------+----------+----------+
|department|Min_Salary|Max_Salary|
+----------+----------+----------+
|   Finance|      1006|      9899|
|  Accounts|      1007|      9890|
|        HR|      1013|      9982|
| Marketing|      1031|      9974|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



In [47]:
# 8. Print the name of employees working in NY stae under Finance department whose bonuses are greater than \
#         an average bonuses of employees in NY state.
avg_employee_bonus_NY = df.filter(df.state == 'NY').groupBy("state").agg(avg('bonus'))
avg_employee_bonus_NY.show()

+-----+------------------+
|state|        avg(bonus)|
+-----+------------------+
|   NY|1251.3468208092486|
+-----+------------------+



In [68]:
avg_NY_bonus_rdd = avg_employee_bonus_NY.rdd.collect()[0]['avg(bonus)']

In [75]:
df.filter((df.state == 'NY') & (df.department == 'Finance') & (df.bonus>avg_NY_bonus_rdd)).show()

+-----------+--------------------+----------+-----+------+---+-----+
|employee_id|       employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
|       1035|       Vivan Sifford|   Finance|   NY|  1129| 35| 1261|
|       1073|      Herder Gallman|   Finance|   NY|  1988| 31| 1402|
|       1082|          Nena Rocha|   Finance|   NY|  3417| 25| 1647|
|       1087|       Leif Lemaster|   Finance|   NY|  8642| 45| 1782|
|       1100|Ellingsworth Meli...|   Finance|   NY|  7845| 32| 1358|
|       1127|        Escoto Gilma|   Finance|   NY|  3426| 41| 1285|
|       1161|     Georgeanna Laub|   Finance|   NY|  2469| 26| 1679|
|       1175|     Durio Tenenbaum|   Finance|   NY|  2253| 42| 1684|
|       1180|       Juliana Grigg|   Finance|   NY|  8178| 42| 1617|
|       1215|        Tiffani Benz|   Finance|   NY|  1665| 41| 1969|
|       1220|          Nitz Ilana|   Finance|   NY|  2443| 50| 1342|
|       1342|   Phylicia Antonina|

In [70]:
# 9. Raise the salaries $500 of all employees whose age is greater than 45.
def incr_salary(age, salary):
    if age>45:
        salary+=500
    return salary

incr_salary_udf = udf(lambda x,y: incr_salary(x,y), IntegerType())

In [71]:
df.withColumn("New_Salary", incr_salary_udf(df.age, df.salary)).show()

+-----------+-------------------+----------+-----+------+---+-----+----------+
|employee_id|      employee_name|department|state|salary|age|bonus|New_Salary|
+-----------+-------------------+----------+-----+------+---+-----+----------+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|      6131|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|      4027|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|      3122|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|      6217|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|      5685|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|      2843|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|      7201|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|      3444|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|      4204|
|       1009|        Imai Hallie|  Accounts|   AK|  

In [72]:
# 10. Create DF of all those employees whose age is greater than 45 and save them in a file
df_age_greater_45 = df.filter(df.age>45)
df_age_greater_45.show()

+-----------+------------------+----------+-----+------+---+-----+
|employee_id|     employee_name|department|state|salary|age|bonus|
+-----------+------------------+----------+-----+------+---+-----+
|       1003|       Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1008| Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1011|  Barringer Escoto|Purchasing|   WA|  1685| 49| 1706|
|       1018|Vankirk Jacquelyne|Purchasing|   NY|  8636| 47| 1192|
|       1025|   Dionne Lemaster|     Sales|   AK|  5134| 48| 1356|
|       1030|        Trena Benz|  Accounts|   NY|  4376| 49| 1624|
|       1039|      Dynes Katlyn|  Accounts|   AK|  3039| 48|  834|
|       1058|      Clune Norene|   Finance|   AK|  1605| 49|  801|
|       1074|      Rocha Dionne|  Accounts|   CA|  3470| 49|  706|
|       1088|       Imai Locust|     Sales|   NY|  9982| 49| 1809|
|       1090| Clemencia Rudolph|   Finance|   NY|  1296| 50| 1209|
|       1099|    Zollner Marvis|Purchasing|   CA|  4230| 50|  

In [78]:
df_age_greater_45.write.csv('OfficeDataProject/out')

Py4JJavaError: An error occurred while calling o529.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:288)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	a